In [2]:
from stock_prediction.data_prep import Data_Prep
from stock_prediction.features_exo import exo_selection

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import os
from sklearn.model_selection import TimeSeriesSplit
import itertools
from sklearn.metrics import mean_absolute_percentage_error as MAPE

<ipython-input-3-217b9ff18a44>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [4]:
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

In [44]:
vinci = Data_Prep('sap', 40)

In [45]:
data_global = vinci.data_prep()

In [46]:
data_exo = vinci.select_features(data_global, Return = True, Log_Return=False, High_Low=False, High_Close=True, Low_Close=True,
                        Volume_Change=False, Period_Volum=False, Annual_Vol=False,
                        Period_Vol=False, Return_Index=False, Volum_Index=False, Relative_Return=False)

In [47]:
#data_exo = exo_selection(data_exo, sp500=True, gold=True, eurusd=True, nasdaq=True, crude=True, vix=True)

In [48]:
data_exo = data_exo[-258:]

In [49]:
data_exo = vinci.Price_Rebase(data_exo)

In [50]:
data_exo.columns

Index(['Date', 'High-Close_SAP.DE', 'Low-Close_SAP.DE', 'Return_SAP.DE_R'], dtype='object')

In [10]:
y_vinci = data_exo['Return_DG.PA_R']
y_sp = data_exo['Return_S&P500_R']
y_usd = data_exo['Return_Usd_R']
y_nasdaq = data_exo['Return_Nasdaq_R']
y_crude = data_exo['Return_Crude_R']
y_vix = data_exo['Vix_No_Rebase']

In [40]:
y_endo = data_exo['Return_SAP.DE_R']
y_exo = np.array(data_exo[['High-Close_SAP.DE', 'Low-Close_SAP.DE']])


In [51]:
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA', FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA', FutureWarning)

range_p = [0,1]
range_d = [0,1]
range_q = [0,1]

grid  = itertools.product(range_p, range_d, range_q)

#empty lists to store results 
orders = []
mapeS = []
aics = []
fold_idxs = []
list_y_pred = []
list_y_test = []
baseline = []
base_score = []
real_value = []

# for all possibilities (pdq) create splits in our TS
for (p,d,q) in grid : 
    order = (p,d,q)
    # make the split of the indexes of our TS\n",
    folds = TimeSeriesSplit(n_splits=30, max_train_size=40, test_size=1)
  
    # fold_idx is the number of the split -- n_splits ?  
    # train_idx are the indexes of the split n°X
    # same for test_idx

    for fold_idx, (train_idx, test_idx) in enumerate(folds.split(y_endo)):
        # we store the number of the split executed
        fold_idxs.append(fold_idx)
        # we retrieve the real data in y 
        #corresponding of the indexes splited 
        y_train = y_endo[train_idx]
        y_exo_train = y_exo[train_idx]
        y_test = y_endo[test_idx]
        y_exo_test = y_exo[test_idx]
        y_true = y_endo[test_idx]
        base = y_train.iloc[-1]
    
        try: 
            # fit our model on y_train of the split n°X
            model = ARIMA(endog=y_train, exog=y_exo_train, order=order).fit()
            # we find our y_pred 
            y_pred = model.forecast(steps=len(y_test), exog=y_exo_test)[0]
            mapeS.append(MAPE(y_pred, y_true))
            orders.append(order)
            aics.append(model.aic)
            list_y_pred.append(y_pred[0])
            list_y_test.append(y_test[test_idx])
            baseline.append(base)
            base_score.append(MAPE(np.array([base]), y_true))
            real_value.append(y_true.values[0])
          
        except:
            pass
     
     
model_results = pd.DataFrame(list(zip(fold_idxs, orders, aics, mapeS, list_y_pred, baseline, base_score, real_value)),
                             columns =['Fold', '(p,d,q)', 'AIC', 'MAPE', 'Pred', 'Baseline', 'base_score', 'True'])


/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnin

...

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:728: RuntimeWarning: divide by zero encountered in true_divide
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:701: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:702: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:470: RuntimeWarning: invalid value encountered in subtract
  dx = ((x0 + h) - x0)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/s

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/ari

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/ari

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/ari

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


In [52]:
model_results.sort_values('AIC').groupby('(p,d,q)').mean()['AIC'].sort_values()

(p,d,q)
(0, 1, 1)    125.866681
(1, 1, 0)    126.039141
(0, 1, 0)    126.077920
(1, 1, 1)    128.368529
(1, 0, 1)    168.169580
(1, 0, 0)    170.806574
(0, 0, 1)    236.468368
(0, 0, 0)    278.465684
Name: AIC, dtype: float64

In [53]:
model_results.sort_values('MAPE').groupby('(p,d,q)').mean()['MAPE'].sort_values()

(p,d,q)
(1, 1, 1)    0.004498
(0, 1, 0)    0.004958
(1, 1, 0)    0.005003
(0, 1, 1)    0.005030
(1, 0, 1)    0.006455
(1, 0, 0)    0.006560
(0, 0, 1)    0.033028
(0, 0, 0)    0.061470
Name: MAPE, dtype: float64

In [ ]:
# check Max Train 

In [ ]:
model_results_max = []


# we search all the possibilities for max_train 
    
for max_train in range(7,35,1) :

    splits = len(y) - max_train

    # reset the lists for next for max_train
    mapeS = []
    base_score = []
    order = (0,1,0)
    # make the split of the indexes of our TS
    folds = TimeSeriesSplit(n_splits=splits, max_train_size=max_train, test_size=1)
    # fold_idx is the number of the split -- n_splits ?  
    # train_idx are the indexes of the split n°X
    # same for test_idx

    for train_idx, test_idx in folds.split(y):

        # we retrieve the real data in y \n",
        #corresponding of the indexes splited 
        y_train = y[train_idx]
        y_test = y[test_idx]
        y_true = y[test_idx]
        base = y_train.iloc[-1]
        #ipdb.set_trace()
        \n",
        # fit our model on y_train of the split n°X
        model = ARIMA(y_train, order=order).fit()
        # we find our y_pred 
        y_pred = model.forecast(len(y_test))[0]
        mapeS.append(MAPE(y_pred, y_true))
        # need np.array to compute mean
        base_score.append(MAPE(np.array([base]), y_true

        mapeS_m = np.array(mapeS)
        base_score_m = np.array(base_score)
        # we store all the results 

        model_results_max.append([max_train, mapeS_m.mean(), base_score_m.mean()

In [ ]:
model_results_max = pd.DataFrame(model_results_max, columns = ['max_train', 'MAPE', 'MAPE_base'])

In [ ]:
# we show the best max_train regarding our MAPE
model_results_max.head().sort_values(by='MAPE')